# Aesthetic Feature Extraction

##### By Aesthetic feature extraction we refer to the aesthetical components of an image this mainly refers to Color, Composition and Texture of an adcreative. This features are extracted on the top of the assumption the peoples are attracted to beauty as a whole eventhough different persons have different perception of beauty.

In [2]:
import os
import numpy as np
import cv2

COLOR

In [7]:
class ImageColor():
    def __init__(self, image_path):
        super(ImageColor, self).__init__()

        # Read in the image
        self.bgr_img = None
        self.hsv_img = None
        self.gray_img = None

        self.h_mean, self.s_mean, self.v_mean = None, None, None
        self.h_std, self.s_std, self.v_std = None, None, None

        # Read in the image
        self.read_in(image_path)

    def update(self, image_path):
        self.read_in(image_path)

    def read_in(self, image_path):
        # This function reads the image into the bgr(blue, green, red), hsv(hue, saturation, value) and gray_scale image.

        self.bgr_img = cv2.imread(image_path)
        self.hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)
        self.gray_img = cv2.imread(image_path, 0)

        # Compute the statics
        self.h_mean, self.s_mean, self.v_mean = np.mean(self.hsv_img, axis=(0, 1))
        self.h_std, self.s_std, self.v_std = np.std(self.hsv_img, axis=(0, 1))

    @staticmethod
    def compute_circular(channel_image):
        A = np.cos(channel_image).sum()
        B = np.sin(channel_image).sum()

        R = 1 - np.sqrt(A ** 2 + B ** 2) / (channel_image.shape[0] * channel_image.shape[1])

        return R

    def compute_hsv_statics(self):
        h_circular = self.compute_circular(self.hsv_img[0])
        v_intensity = np.sqrt((self.hsv_img[-1] ** 2).mean())

        return [self.s_mean, self.s_std, self.v_std, h_circular, v_intensity]

    def compute_emotion_based(self):
        valence = 0.69 * self.v_mean + 0.22 * self.s_mean
        arousal = -0.31 * self.v_mean + 0.6 * self.s_mean
        dominance = -0.76 * self.v_mean + 0.32 * self.s_mean

        return [valence, arousal, dominance]
    
    def compute_color_diversity(self):
        """Adapted from
        https://github.com/yilangpeng/computational-aesthetics/blob/27ff52b47b880bd46a14a7b062a4dde69b6a9988/basic.py#L46-L56
        """
        rgb = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2RGB).astype(float)

        l_rgbR, l_rgbG, l_rgbB = cv2.split(rgb)
        l_rg = l_rgbR - l_rgbG
        l_yb = 0.5 * l_rgbR + 0.5 * l_rgbG - l_rgbB

        rg_sd = np.std(l_rg)
        rg_mean = np.mean(l_rg)
        yb_sd = np.std(l_yb)
        yb_mean = np.mean(l_yb)

        rg_yb_sd = (rg_sd ** 2 + yb_sd ** 2) ** 0.5
        rg_yb_mean = (rg_mean ** 2 + yb_mean ** 2) ** 0.5
        colorful = rg_yb_sd + (rg_yb_mean * 0.3)

        return [colorful]

    def compute_color_info(self):
        hsv_res = self.compute_hsv_statics()
        emotion_res = self.compute_emotion_based()
        color_div_res = self.compute_color_diversity()

        return hsv_res + emotion_res + color_div_res

In [14]:
image = ImageColor("/home/michael_getachew/creative-optimization/creative-optimisation-cv/data/Challenge_Data/Assets/0a59be2e7dd53d6de11a10ce3649c081/_preview.png")
res_diversity = image.compute_color_diversity()
res_emotion = image.compute_emotion_based()
res_hsvstat = image.compute_hsv_statics()
print(res_diversity)
print(res_emotion)

[27.311312694085938]
[137.71552599999998, -34.069985111111116, -129.31574844444444]


COMPOSITION

In [1]:
import functools
import operator
import numpy as np
import pywt
from sklearn.cluster import (
    MeanShift,
    estimate_bandwidth
)
import cv2

In [3]:
class Composition():
    def __init__(self, image_path):
        super(Composition, self).__init__()
        self.bgr_img = None
        self.hsv_img = None
        self.gray_img = None
        self.read_in(image_path)

    def compute_edge_pixels(self,
                            blur_size: int = 3,
                            ratio_low=0.4, ratio_up=0.8):
        """Adapted from
        https://github.com/yilangpeng/computational-aesthetics/blob/master/edge.py
        """
        h, w = self.bgr_img.shape[:2]
        blur_img = cv2.GaussianBlur(self.gray_img, (blur_size, blur_size), 0)

        thresh_low = min(100, np.quantile(blur_img, q=ratio_low))
        thresh_up = max(200, np.quantile(blur_img, q=ratio_up))

        edges_img = cv2.Canny(blur_img,
                              threshold1=thresh_low,
                              threshold2=thresh_up)
        num_edges = np.count_nonzero(edges_img) / (h * w)

        return [num_edges]

    def update(self, image_path):
        self.read_in(image_path)

    def read_in(self, image_path):
        self.bgr_img = cv2.imread(image_path)
        self.hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)
        self.gray_img = cv2.imread(image_path, 0)

    def compute_level_of_details(self,
                                 quantile=0.2,
                                 n_samples=3000,
                                 thresh=0.05):
        # Using quick shift segmentation
        rgb_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2RGB)

        # Flatten the image
        flat_image = rgb_img.reshape(-1, 3)
        flat_image = np.float32(flat_image)

        bandwidth = estimate_bandwidth(flat_image,
                                       quantile=quantile,
                                       n_samples=n_samples)
        mean_shift = MeanShift(bandwidth, bin_seeding=True)
        mean_shift.fit(flat_image)
        image_labels = mean_shift.labels_

        h, w = rgb_img.shape[:2]
        unique_labels, unique_counts = np.unique(image_labels,
                                                 return_counts=True)

        # Remove small region of image
        mean_thresh = (h * w) * thresh
        unique_labels = unique_labels[unique_counts > mean_thresh]
        unique_counts = unique_counts[unique_counts > mean_thresh]

        num_seg = len(unique_labels)
        average_size = unique_counts.mean() / (h * w)

        return [num_seg, average_size]

    @staticmethod
    def compute_channel_depth_of_field(channel):
        level_wanted = channel[1]
        h, w = level_wanted[0].shape[:2]

        # Create blank image to include all channel
        blank_image = np.zeros((h, w, 3))

        for idx, level_wanted_matrix in enumerate(level_wanted):
            blank_image[..., idx] = np.abs(level_wanted_matrix)

        
        # Compute M6, M7, M10, M11
        start_x, end_x = int(h / 4), int(3 * h / 4)
        start_y, end_y = int(w / 4), int(3 * w / 4)

        dof_channel = np.sum(blank_image[start_x:end_x, start_y:end_y, :]) / np.sum(blank_image)

        return dof_channel

    def compute_depth_of_field(self):
        # Process for the H channel
        h_wavelet = pywt.wavedec2(self.hsv_img[..., 0], mode="periodization",
                                  wavelet="db3", level=3)
        h_dof = self.compute_channel_depth_of_field(h_wavelet)

        # Process for the S channel
        s_wavelet = pywt.wavedec2(self.hsv_img[..., 1], mode="periodization",
                                  wavelet="db3", level=3)
        s_dof = self.compute_channel_depth_of_field(s_wavelet)

        # Process for the V channel
        v_wavelet = pywt.wavedec2(self.hsv_img[..., 2], mode="periodization",
                                  wavelet="db3", level=3)
        v_dof = self.compute_channel_depth_of_field(v_wavelet)

        return [h_dof, s_dof, v_dof]

    def compute_rule_of_third(self):
        h, w = self.bgr_img.shape[:2]

        # Convert to hsv
        hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)

        # Set the end points of image
        start_h, end_h = int(h / 3), int(2 * h / 3)
        start_w, end_w = int(w / 3), int(2 * w / 3)
        center_image = hsv_img[start_h:end_h, start_w:end_w]

        # Compute the mean of saturation and value
        s_mean = np.mean(center_image[..., 1])
        v_mean = np.mean(center_image[..., 2])

        return [s_mean, v_mean]

    
    

In [6]:
composition = Composition("/home/michael_getachew/creative-optimization/creative-optimisation-cv/data/Challenge_Data/Assets/0a22f881b77f00220f2034c21a18b854/_preview.png")
count_edges = composition.compute_depth_of_field()
edge_res = composition.compute_edge_pixels()
depth_of_field = composition.compute_depth_of_field()
rule_of_thirds_res = composition.compute_rule_of_third()
print(count_edges)
print(edge_res)
print(depth_of_field)
print(rule_of_thirds_res)

[0.24078261539783, 0.36778763751914645, 0.33361000005577574]
[0.04909]
[0.24078261539783, 0.36778763751914645, 0.33361000005577574]
[71.72994011976049, 75.76410179640719]


Texture

In [7]:
import math
import cv2
import numpy as np
import pywt
import skimage.measure
import skimage.feature
import skimage.morphology
import skimage.filters.rank

In [8]:
class Texture():
    def __init__(self, image_path):
        super(Texture, self).__init__()
        self.bgr_img = None
        self.hsv_img = None
        self.gray_img = None
        self.read_in(image_path)

    def compute_entropy(self):
        entropy = skimage.filters.rank.entropy(image=self.gray_img,
                                               selem=skimage.morphology.square(9))
        entropy = entropy.mean()
        return [entropy]
